# INSTALLING DEPENDENCIES
This cell installs all the necessary dependencies.


In [ ]:
!pip install fastapi uvicorn python-multipart
!pip install google-generativeai groq
!pip install PyMuPDF pdfplumber python-docx
!pip install nest-asyncio pyngrok
!apt-get install -y poppler-utils



In [ ]:
# For Colab deployment
import nest_asyncio
nest_asyncio.apply()


# Importing necessaary modules


In [ ]:
import os
import re
import json
import tempfile
from typing import Dict, List, Optional, Any
from pathlib import Path
import logging

# FastAPI
from fastapi import FastAPI, File, UploadFile, Form, HTTPException
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
import uvicorn

# Document Processing
import fitz  # PyMuPDF
import pdfplumber
from docx import Document

# AI APIs
try:
    import google.generativeai as genai
    GEMINI_AVAILABLE = True
except ImportError:
    GEMINI_AVAILABLE = False

try:
    from groq import Groq
    GROQ_AVAILABLE = True
except ImportError:
    GROQ_AVAILABLE = False


In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Environment Configuration
MAX_FILE_SIZE = 10 * 1024 * 1024  # 10MB
SUPPORTED_FORMATS = [".pdf", ".docx", ".txt"]

# Global variables
_global_api_key = None
_current_provider = None  # 'gemini' or 'groq'


#SETTING API KEYS
You can get the api keys free and insert it here (you can use gemini or groq).

In [ ]:
def set_gemini_key(api_key: str):
    """
    Set Google Gemini API key (FREE).

    Get your free key at: https://makersuite.google.com/app/apikey

    Args:
        api_key: Your Gemini API key
    """
    global _global_api_key, _current_provider
    if not GEMINI_AVAILABLE:
        print("❌ Google GenerativeAI not installed!")
        print("   Run: !pip install google-generativeai")
        return False

    try:
        genai.configure(api_key=api_key)
        # Test the key
        model = genai.GenerativeModel("gemini-2.5-flash")
        response = model.generate_content("test")

        _global_api_key = api_key
        _current_provider = 'gemini'
        os.environ['GEMINI_API_KEY'] = api_key
        print("✅ Gemini API key set successfully!")
        print("🚀 Provider: Google Gemini (FREE)")
        return True
    except Exception as e:
        print(f"❌ Gemini API key verification failed: {e}")
        return False

def set_groq_key(api_key: str):
    """
    Set Groq API key (FREE, very fast).

    Get your free key at: https://console.groq.com/keys

    Args:
        api_key: Your Groq API key
    """
    global _global_api_key, _current_provider
    if not GROQ_AVAILABLE:
        print("❌ Groq not installed!")
        print("   Run: !pip install groq")
        return False

    try:
        client = Groq(api_key=api_key)
        # Test the key
        response = client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[{"role": "user", "content": "test"}],
            max_tokens=5
        )

        _global_api_key = api_key
        _current_provider = 'groq'
        os.environ['GROQ_API_KEY'] = api_key
        print("✅ Groq API key set successfully!")
        print("🚀 Provider: Groq (FREE & Fast)")
        return True
    except Exception as e:
        print(f"❌ Groq API key verification failed: {e}")
        return False

def verify_api_setup():
    """Verify that an API key is properly configured."""
    if _current_provider is None:
        print("❌ No API key configured!")
        print("\n📝 Choose a FREE API provider:")
        print("\n1. Google Gemini (RECOMMENDED - Completely FREE)")
        print("   • Get key: https://makersuite.google.com/app/apikey")
        print("   • Setup: set_gemini_key('your-key-here')")
        print("\n2. Groq (FREE & Very Fast)")
        print("   • Get key: https://console.groq.com/keys")
        print("   • Setup: set_groq_key('your-key-here')")
        return False

    print(f"✅ API configured: {_current_provider.upper()}")
    return True



#Resume Parsing Logic
Handles extraction and parsing of resume content from various file formats.

In [ ]:
class ResumeParser:
    """Handles extraction and parsing of resume content from various file formats."""

    @staticmethod
    def extract_text_from_pdf(file_path: str) -> str:
        """Extract text from PDF using PyMuPDF with fallback to pdfplumber."""
        try:
            text = ""
            with fitz.open(file_path) as doc:
                for page in doc:
                    text += page.get_text()

            if text.strip():
                return text

            logger.info("Falling back to pdfplumber for PDF extraction")
            with pdfplumber.open(file_path) as pdf:
                text = "\n".join([page.extract_text() or "" for page in pdf.pages])

            return text
        except Exception as e:
            logger.error(f"PDF extraction error: {e}")
            raise ValueError(f"Failed to extract text from PDF: {str(e)}")

    @staticmethod
    def extract_text_from_docx(file_path: str) -> str:
        """Extract text from DOCX file."""
        try:
            doc = Document(file_path)
            text = "\n".join([paragraph.text for paragraph in doc.paragraphs])

            for table in doc.tables:
                for row in table.rows:
                    for cell in row.cells:
                        text += "\n" + cell.text

            return text
        except Exception as e:
            logger.error(f"DOCX extraction error: {e}")
            raise ValueError(f"Failed to extract text from DOCX: {str(e)}")

    @staticmethod
    def extract_text_from_txt(file_path: str) -> str:
        """Extract text from TXT file with encoding detection."""
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                return f.read()
        except UnicodeDecodeError:
            try:
                with open(file_path, 'r', encoding='latin-1') as f:
                    return f.read()
            except Exception as e:
                logger.error(f"TXT extraction error: {e}")
                raise ValueError(f"Failed to extract text from TXT: {str(e)}")

    @staticmethod
    def clean_text(text: str) -> str:
        """Clean and preprocess extracted text."""
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'[^\w\s.,;:()\-@/+#]', '', text)
        text = re.sub(r'\n+', '\n', text)
        return text.strip()

    @classmethod
    def parse_resume(cls, file_path: str, file_extension: str) -> Dict[str, Any]:
        """Main parsing function that routes to appropriate extractor."""
        logger.info(f"Parsing resume: {file_path} ({file_extension})")

        if file_extension == '.pdf':
            raw_text = cls.extract_text_from_pdf(file_path)
        elif file_extension == '.docx':
            raw_text = cls.extract_text_from_docx(file_path)
        elif file_extension == '.txt':
            raw_text = cls.extract_text_from_txt(file_path)
        else:
            raise ValueError(f"Unsupported file format: {file_extension}")

        if not raw_text or len(raw_text.strip()) < 50:
            raise ValueError("Insufficient text extracted from resume.")

        cleaned_text = cls.clean_text(raw_text)

        return {
            "raw_text": raw_text,
            "cleaned_text": cleaned_text,
            "file_type": file_extension
        }


#Resume Analyzing Logic
Handles resume anlaysis using free API.


In [ ]:
class FreeAIResumeAnalyzer:
    """
    Handles resume analysis using FREE AI APIs (Gemini or Groq).
    """

    def __init__(self, provider: str, api_key: str):
        """
        Initialize the analyzer with chosen provider.

        Args:
            provider: 'gemini' or 'groq'
            api_key: API key for the chosen provider
        """
        self.provider = provider

        if provider == 'gemini':
            genai.configure(api_key=api_key)
            self.model = genai.GenerativeModel('gemini-2.5-flash')
        elif provider == 'groq':
            self.client = Groq(api_key=api_key)
            self.model_name = "llama-3.1-70b-versatile"  # Best free model
        else:
            raise ValueError(f"Unsupported provider: {provider}")

    def _generate_content(self, prompt: str, max_tokens: int = 2000) -> str:
        """Generate content using the configured provider."""
        try:
            if self.provider == 'gemini':
                response = self.model.generate_content(prompt)
                return response.text

            elif self.provider == 'groq':
                response = self.client.chat.completions.create(
                    model=self.model_name,
                    messages=[
                        {"role": "system", "content": "You are an expert resume analyst and career coach."},
                        {"role": "user", "content": prompt}
                    ],
                    temperature=0.4,
                    max_tokens=max_tokens
                )
                return response.choices[0].message.content
        except Exception as e:
            logger.error(f"Content generation error: {e}")
            raise

    def extract_structured_info(self, resume_text: str) -> Dict[str, Any]:
        """Extract structured information from resume."""
        prompt = f"""Extract structured information from this resume. Return ONLY valid JSON with these exact fields:
{{
    "name": "full name or 'Not Found'",
    "email": "email or 'Not Found'",
    "phone": "phone or 'Not Found'",
    "skills": ["skill1", "skill2", ...],
    "experience": [
        {{"title": "job title", "company": "company name", "duration": "time period", "description": "brief description"}}
    ],
    "education": [
        {{"degree": "degree name", "institution": "school name", "year": "graduation year"}}
    ],
    "projects": ["project1", "project2", ...]
}}

Resume:
{resume_text[:4000]}

Return ONLY the JSON, no other text."""

        try:
            result = self._generate_content(prompt, max_tokens=1500)

            # Clean response
            if result.startswith("```"):
                result = re.sub(r'```json\s*|\s*```', '', result).strip()

            return json.loads(result)
        except json.JSONDecodeError as e:
            logger.error(f"JSON parsing error: {e}")
            return self._get_default_structure()
        except Exception as e:
            logger.error(f"Extraction error: {e}")
            return self._get_default_structure()

    def analyze_resume(self, resume_text: str, job_description: str,
                      structured_info: Dict[str, Any]) -> Dict[str, Any]:
        """Comprehensive resume analysis."""
        prompt = f"""Analyze this resume against the job description. Provide a comprehensive evaluation.

JOB DESCRIPTION:
{job_description[:2000]}

RESUME:
{resume_text[:3000]}

EXTRACTED INFO:
Skills: {', '.join(structured_info.get('skills', [])[:20])}
Experience: {len(structured_info.get('experience', []))} positions

Provide analysis in this EXACT JSON format:
{{
    "skills_analysis": {{
        "score": 0-100,
        "matching_skills": ["skill1", "skill2"],
        "missing_skills": ["skill1", "skill2"],
        "recommendations": ["specific suggestion 1", "specific suggestion 2"]
    }},
    "experience_analysis": {{
        "score": 0-100,
        "strengths": ["strength1", "strength2"],
        "weaknesses": ["weakness1", "weakness2"],
        "recommendations": ["specific suggestion 1", "specific suggestion 2"]
    }},
    "language_analysis": {{
        "score": 0-100,
        "grammar_quality": "excellent/good/fair/poor",
        "readability": "excellent/good/fair/poor",
        "professionalism": "excellent/good/fair/poor",
        "recommendations": ["specific suggestion 1", "specific suggestion 2"]
    }},
    "formatting_analysis": {{
        "score": 0-100,
        "ats_friendly": true/false,
        "structure_quality": "excellent/good/fair/poor",
        "recommendations": ["specific suggestion 1", "specific suggestion 2"]
    }},
    "job_fit_score": 0-100,
    "overall_recommendations": ["high-priority suggestion 1", "suggestion 2", "suggestion 3"]
}}

Be specific, actionable, and honest. Return ONLY valid JSON."""

        try:
            result = self._generate_content(prompt, max_tokens=2500)

            # Clean response
            if result.startswith("```"):
                result = re.sub(r'```json\s*|\s*```', '', result).strip()

            analysis = json.loads(result)
            return self._validate_analysis(analysis)

        except Exception as e:
            logger.error(f"Analysis error: {e}")
            return self._get_default_analysis()

    def generate_improved_resume(self, resume_text: str, job_description: str,
                                analysis: Dict[str, Any]) -> str:
        """Generate an improved version of the resume."""
        recommendations = analysis.get('overall_recommendations', [])
        missing_skills = analysis.get('skills_analysis', {}).get('missing_skills', [])

        prompt = f"""Improve this resume based on the analysis and job requirements.

ORIGINAL RESUME:
{resume_text[:3000]}

JOB DESCRIPTION:
{job_description[:1500]}

KEY RECOMMENDATIONS:
{chr(10).join([f"- {rec}" for rec in recommendations[:5]])}

MISSING SKILLS TO HIGHLIGHT:
{', '.join(missing_skills[:10])}

Create an improved version that:
1. Enhances clarity and impact
2. Adds quantifiable achievements
3. Improves keyword optimization
4. Maintains all original information
5. Uses strong action verbs
6. Improves formatting

Return the improved resume in clean, professional format."""

        try:
            return self._generate_content(prompt, max_tokens=3000)
        except Exception as e:
            logger.error(f"Resume generation error: {e}")
            return "Error generating improved resume. Please try again."

    @staticmethod
    def _get_default_structure() -> Dict[str, Any]:
        """Return default structure when extraction fails."""
        return {
            "name": "Not Found",
            "email": "Not Found",
            "phone": "Not Found",
            "skills": [],
            "experience": [],
            "education": [],
            "projects": []
        }

    @staticmethod
    def _get_default_analysis() -> Dict[str, Any]:
        """Return default analysis when analysis fails."""
        return {
            "skills_analysis": {"score": 50, "matching_skills": [], "missing_skills": [], "recommendations": []},
            "experience_analysis": {"score": 50, "strengths": [], "weaknesses": [], "recommendations": []},
            "language_analysis": {"score": 50, "grammar_quality": "unknown", "readability": "unknown", "professionalism": "unknown", "recommendations": []},
            "formatting_analysis": {"score": 50, "ats_friendly": False, "structure_quality": "unknown", "recommendations": []},
            "job_fit_score": 50,
            "overall_recommendations": ["Analysis failed. Please try again."]
        }

    @staticmethod
    def _validate_analysis(analysis: Dict[str, Any]) -> Dict[str, Any]:
        """Validate and sanitize analysis results."""
        for key in ['skills_analysis', 'experience_analysis', 'language_analysis', 'formatting_analysis']:
            if key in analysis and 'score' in analysis[key]:
                score = analysis[key]['score']
                analysis[key]['score'] = max(0, min(100, int(score)))

        if 'job_fit_score' in analysis:
            analysis['job_fit_score'] = max(0, min(100, int(analysis['job_fit_score'])))

        return analysis


#Scoring Engine
It calculates and give scores according to the provided job description.


In [ ]:
class ScoringEngine:
    """Calculates final scores and compiles comprehensive results."""

    @staticmethod
    def calculate_overall_score(analysis: Dict[str, Any]) -> int:
        """Calculate weighted overall score from component scores."""
        weights = {
            'skills_score': 0.30,
            'experience_score': 0.25,
            'language_score': 0.15,
            'formatting_score': 0.10,
            'job_fit_score': 0.20
        }

        skills_score = analysis.get('skills_analysis', {}).get('score', 50)
        experience_score = analysis.get('experience_analysis', {}).get('score', 50)
        language_score = analysis.get('language_analysis', {}).get('score', 50)
        formatting_score = analysis.get('formatting_analysis', {}).get('score', 50)
        job_fit_score = analysis.get('job_fit_score', 50)

        overall = (
            skills_score * weights['skills_score'] +
            experience_score * weights['experience_score'] +
            language_score * weights['language_score'] +
            formatting_score * weights['formatting_score'] +
            job_fit_score * weights['job_fit_score']
        )

        return int(round(overall))

    @staticmethod
    def compile_suggestions(analysis: Dict[str, Any]) -> List[str]:
        """Compile all suggestions into prioritized list."""
        suggestions = []
        suggestions.extend(analysis.get('overall_recommendations', [])[:3])

        for key in ['skills_analysis', 'experience_analysis', 'language_analysis', 'formatting_analysis']:
            if key in analysis and 'recommendations' in analysis[key]:
                suggestions.extend(analysis[key]['recommendations'][:2])

        seen = set()
        unique_suggestions = []
        for suggestion in suggestions:
            if suggestion.lower() not in seen:
                seen.add(suggestion.lower())
                unique_suggestions.append(suggestion)

        return unique_suggestions[:10]

    @classmethod
    def generate_final_output(cls, analysis: Dict[str, Any],
                            structured_info: Dict[str, Any]) -> Dict[str, Any]:
        """Generate final structured output for API response."""
        overall_score = cls.calculate_overall_score(analysis)
        suggestions = cls.compile_suggestions(analysis)

        return {
            "overall_score": overall_score,
            "skills_score": analysis.get('skills_analysis', {}).get('score', 50),
            "experience_score": analysis.get('experience_analysis', {}).get('score', 50),
            "language_score": analysis.get('language_analysis', {}).get('score', 50),
            "formatting_score": analysis.get('formatting_analysis', {}).get('score', 50),
            "job_fit_score": analysis.get('job_fit_score', 50),
            "suggestions": suggestions,
            "detailed_analysis": {
                "skills": analysis.get('skills_analysis', {}),
                "experience": analysis.get('experience_analysis', {}),
                "language": analysis.get('language_analysis', {}),
                "formatting": analysis.get('formatting_analysis', {})
            },
            "extracted_info": structured_info,
            "ai_provider": _current_provider
        }


#FAST API
Provides an ui for the user.

In [ ]:
app = FastAPI(
    title="AI Resume Analyzer (FREE APIs)",
    description="Resume analysis using FREE AI APIs (Gemini/Groq)",
    version="2.0.0"
)

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

def get_analyzer():
    """Get or create the AI analyzer instance."""
    if _current_provider is None or _global_api_key is None:
        raise HTTPException(
            status_code=500,
            detail="API not configured. Please set up Gemini or Groq API key first."
        )
    return FreeAIResumeAnalyzer(_current_provider, _global_api_key)

@app.post("/analyze_resume/")
async def analyze_resume(
    resume: UploadFile = File(...),
    job_description: str = Form(...),
    generate_improved: bool = Form(False)
):
    """Analyze a resume against a job description using FREE AI APIs."""
    temp_file = None

    try:
        analyzer = get_analyzer()

        if not resume.filename:
            raise HTTPException(status_code=400, detail="No file provided")

        file_extension = Path(resume.filename).suffix.lower()
        if file_extension not in SUPPORTED_FORMATS:
            raise HTTPException(
                status_code=400,
                detail=f"Unsupported format. Supported: {', '.join(SUPPORTED_FORMATS)}"
            )

        if not job_description or len(job_description.strip()) < 20:
            raise HTTPException(
                status_code=400,
                detail="Job description must be at least 20 characters"
            )

        with tempfile.NamedTemporaryFile(delete=False, suffix=file_extension) as temp_file:
            content = await resume.read()

            if len(content) > MAX_FILE_SIZE:
                raise HTTPException(
                    status_code=400,
                    detail=f"File too large. Max: {MAX_FILE_SIZE / 1024 / 1024}MB"
                )

            temp_file.write(content)
            temp_file_path = temp_file.name

        logger.info("Parsing resume...")
        parsed_data = ResumeParser.parse_resume(temp_file_path, file_extension)

        logger.info("Extracting structured information...")
        structured_info = analyzer.extract_structured_info(parsed_data['cleaned_text'])

        logger.info("Analyzing resume with AI...")
        analysis = analyzer.analyze_resume(
            parsed_data['cleaned_text'],
            job_description,
            structured_info
        )

        logger.info("Compiling results...")
        final_output = ScoringEngine.generate_final_output(analysis, structured_info)

        if generate_improved:
            logger.info("Generating improved resume...")
            improved_resume = analyzer.generate_improved_resume(
                parsed_data['raw_text'],
                job_description,
                analysis
            )
            final_output['improved_resume'] = improved_resume

        return JSONResponse(content=final_output)

    except ValueError as e:
        raise HTTPException(status_code=400, detail=str(e))
    except Exception as e:
        logger.error(f"Unexpected error: {e}")
        raise HTTPException(status_code=500, detail=f"Internal error: {str(e)}")
    finally:
        if temp_file and os.path.exists(temp_file.name):
            try:
                os.unlink(temp_file.name)
            except Exception as e:
                logger.warning(f"Failed to delete temp file: {e}")

@app.get("/health")
async def health_check():
    """Health check endpoint."""
    return {
        "status": "healthy",
        "version": "2.0.0",
        "provider": _current_provider or "not_configured",
        "free_api": True
    }

@app.get("/")
async def root():
    """Root endpoint with API information."""
    return {
        "message": "AI Resume Analyzer API (FREE)",
        "version": "2.0.0",
        "provider": _current_provider or "not_configured",
        "endpoints": {
            "/analyze_resume/": "POST - Analyze resume",
            "/health": "GET - Health check",
            "/docs": "GET - API documentation"
        }
    }


#Running on server using ngrok
Running fastapi in colab or jupyter notebook without interfering in running other cells operation.

In [ ]:
import threading
import uvicorn
import nest_asyncio

def run_colab_server(port: int = 8001, use_ngrok: bool = False, ngrok_token: str = None):
    """Run FastAPI server in Colab without blocking other cells."""
    nest_asyncio.apply()

    if use_ngrok:
        from pyngrok import ngrok
        if ngrok_token:
            ngrok.set_auth_token(ngrok_token)
        public_url = ngrok.connect(port)
        print(f"📡 Public URL: {public_url}")
        print(f"📚 API Docs: {public_url}/docs")
    else:
        print(f"🚀 Running locally on port {port} (Colab VM)")
        print(f"📚 Docs: http://localhost:{port}/docs")

    # Run server in background thread
    def start():
        uvicorn.run(app, host="0.0.0.0", port=port, log_level="info")

    thread = threading.Thread(target=start, daemon=True)
    thread.start()


In [ ]:
if __name__ == "__main__":
    try:
        import google.colab
        IN_COLAB = True
    except ImportError:
        IN_COLAB = False

    if IN_COLAB:
        print("🔬 Running in Google Colab mode")
        print("\n📝 Setup Instructions:")
        print("1. Get FREE API key:")
        print("   • Gemini: https://makersuite.google.com/app/apikey")
        print("   • Groq: https://console.groq.com/keys")
        print("\n2. Set your key:")
        print("   set_gemini_key('your-key-here')  # OR")
        print("   set_groq_key('your-key-here')")
        print("\n3. Start server:")
        print("   run_colab_server()")
    else:
        print("💻 Running in local mode")
        if not verify_api_setup():
            print("\nPlease set up an API key first!")
        else:
            uvicorn.run(app, host="0.0.0.0", port=8000, reload=True)


#Gemini API key
Provide the api key that you have copied from google ai studio or groq api .

In [ ]:
# Replace with your actual
set_gemini_key('Your_gemini_api_key')

#Checking the API

In [ ]:


# Check if everything is ready
if verify_api_setup():
    print("🎉 Ready to analyze resumes!")
else:
    print("⚠️ Please set an API key first")


In [ ]:

#Killing all the previous running servers
!killall ngrok

#**Running** the server using ngrok

In [ ]:
run_colab_server(port=8000, use_ngrok=True)

#Checking the resume against the job description provided and giving details,scores and suggestions in an json file.

In [ ]:
import requests
from google.colab import files
import json

# Upload your resume
print("📄 Upload your resume (PDF, DOCX, or TXT):")
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Replace with YOUR ngrok URL from Step 6
API_URL = "https://1cff299cd342.ngrok-free.app"

# Prepare the request
url = f"{API_URL}/analyze_resume/"
files_data = {'resume': open(filename, 'rb')}
form_data = {
    'job_description': '''
    Senior Software Engineer

    We're looking for an experienced software engineer to join our team.

    Required Skills:
    - 5+ years of software development experience
    - Strong programming skills (Python, Java, or JavaScript)
    - Experience with web frameworks (Django, Flask, React, Angular)
    - Database knowledge (SQL, NoSQL)
    - RESTful API design
    - Git version control
    - Agile methodology

    Preferred Skills:
    - Cloud platforms (AWS, GCP, Azure)
    - Docker and Kubernetes
    - CI/CD pipelines
    - Machine Learning basics
    - Microservices architecture

    Responsibilities:
    - Design and develop scalable applications
    - Write clean, maintainable code
    - Collaborate with cross-functional teams
    - Mentor junior developers
    - Participate in code reviews
    ''',
    'generate_improved': 'True'  # Set to 'true' for improved resume
}

# Analyze the resume
print("\n⏳ Analyzing your resume with AI...")
response = requests.post(url, files=files_data, data=form_data)

if response.status_code == 200:
    result = response.json()

    # Display beautiful results
    print("\n" + "="*70)
    print("📊 RESUME ANALYSIS RESULTS")
    print("="*70)
    print(f"\n🎯 Overall Score: {result['overall_score']}/100")
    print(f"💡 Skills Score: {result['skills_score']}/100")
    print(f"💼 Experience Score: {result['experience_score']}/100")
    print(f"✍️  Language Score: {result['language_score']}/100")
    print(f"📋 Formatting Score: {result['formatting_score']}/100")
    print(f"🎪 Job Fit Score: {result['job_fit_score']}/100")
    print(f"🤖 AI Provider: {result['ai_provider'].upper()}")

    print("\n" + "="*70)
    print("💡 TOP SUGGESTIONS FOR IMPROVEMENT:")
    print("="*70)
    for i, suggestion in enumerate(result['suggestions'], 1):
        print(f"\n{i}. {suggestion}")

    # Show detailed analysis
    print("\n" + "="*70)
    print("🔍 DETAILED SKILLS ANALYSIS:")
    print("="*70)
    skills = result['detailed_analysis']['skills']
    print(f"\n✅ Matching Skills: {', '.join(skills.get('matching_skills', [])[:5])}")
    print(f"❌ Missing Skills: {', '.join(skills.get('missing_skills', [])[:5])}")

    # Save results
    with open('resume_analysis.json', 'w') as f:
        json.dump(result, f, indent=2)
    print("\n\n✅ Full results saved to 'resume_analysis.json'")

    # Download results
    files.download('resume_analysis.json')

else:
    print(f"❌ Error: {response.status_code}")
    print(response.text)



#A feature for the one who wants to improve thier resume for a particular job domain and also improving the quality of the resume.

In [ ]:
import requests
from google.colab import files
import json

# Upload your resume
print("📄 Upload your resume (PDF, DOCX, or TXT):")
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Replace with YOUR ngrok URL from Step 6
API_URL = "https://1cff299cd342.ngrok-free.app"

# Prepare the request
url = f"{API_URL}/analyze_resume/"
files_data = {'resume': open(filename, 'rb')}
form_data = {
    'job_description': '''
    Senior Software Engineer

    We're looking for an experienced software engineer to join our team.

    Required Skills:
    - 5+ years of software development experience
    - Strong programming skills (Python, Java, or JavaScript)
    - Experience with web frameworks (Django, Flask, React, Angular)
    - Database knowledge (SQL, NoSQL)
    - RESTful API design
    - Git version control
    - Agile methodology

    Preferred Skills:
    - Cloud platforms (AWS, GCP, Azure)
    - Docker and Kubernetes
    - CI/CD pipelines
    - Machine Learning basics
    - Microservices architecture

    Responsibilities:
    - Design and develop scalable applications
    - Write clean, maintainable code
    - Collaborate with cross-functional teams
    - Mentor junior developers
    - Participate in code reviews
    ''',
    'generate_improved': 'True'  # Set to 'true' for improved resume
}

# After getting results:
if 'improved_resume' in result:
    print("\n" + "="*70)
    print("✨ IMPROVED RESUME VERSION:")
    print("="*70)
    print(result['improved_resume'])

    # Save improved resume
    with open('improved_resume.txt', 'w') as f:
        f.write(result['improved_resume'])
    print("\n✅ Improved resume saved to 'improved_resume.txt'")
    files.download('improved_resume.txt')
